# Atom feed of highly tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least five times as atom feed, grouped by subject area.

In [1]:
import sys
sys.path.append('../')

import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator

In [2]:
subject_areas = ['covid', 'microbiology', 'neuroscience', 'molecular-biology', 'bioinformatics', 'genomics', 'genetics', 'developmental-biology', 'plant-biology', 'evolutionary-biology', 'ecology', 'immunology', 'cancer-biology', 'cell-biology', 'biophysics']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
subsets['microbiology'] = preprints[(preprints['subject-area'] == 'Microbiology') & (preprints['covid'] == False)]
subsets['neuroscience'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
subsets['molecular-biology'] = preprints[(preprints['subject-area'] == 'Molecular Biology') & (preprints['covid'] == False)]
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
subsets['genomics'] = preprints[(preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') & (preprints['covid'] == False)]
subsets['developmental-biology'] = preprints[(preprints['subject-area'] == 'Developmental Biology') & (preprints['covid'] == False)]
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['immunology'] = preprints[(preprints['subject-area'] == 'Immunology') & (preprints['covid'] == False)]
subsets['cancer-biology'] = preprints[(preprints['subject-area'] == 'Cancer Biology') & (preprints['covid'] == False)]
subsets['cell-biology'] = preprints[(preprints['subject-area'] == 'Cell Biology') & (preprints['covid'] == False)]
subsets['biophysics'] = preprints[(preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
subsets['cell-biology'].head(50)

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
4,10.1101/2021.06.30.450524,226,bioRxiv,Cell Biology,False,Horizontal Transfer of Histone H3 by Mammalian...,"[{'name': 'Parker L. Sulkowski'}, {'name': 'Hi...","<p>Histones are small, highly basic, nuclear p...",2021-06-30
13,10.1101/2021.06.28.450262,37,bioRxiv,Cell Biology,False,Actin force generation in vesicle formation: m...,"[{'name': 'Daniel Serwas'}, {'name': 'Matthew ...",<p>Actin filament assembly provides force duri...,2021-06-29
17,10.1101/2021.07.02.450855,30,bioRxiv,Cell Biology,False,Integrated light and electron microscopy conti...,"[{'name': 'Edoardo Dimprima'}, {'name': 'Marta...","<p>3D cell cultures, in particular organoids, ...",2021-07-03
18,10.1101/2021.06.30.450416,25,bioRxiv,Cell Biology,False,"CellProfiler 4: Improvements in Speed, Utility...","[{'name': 'David R. Stirling'}, {'name': 'Madi...","<p>CellProfiler is a free, open source image a...",2021-06-30
27,10.1101/2021.07.02.450924,18,bioRxiv,Cell Biology,False,Mitochondrial dysfunction triggers secretion o...,"[{'name': 'Kimberly A. Jett'}, {'name': 'Zaker...",<p>Signaling circuits crucial to systemic phys...,2021-07-03


In [3]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular BioRxiv {subject_area.replace("_", " ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')